#### OMDB AND TasteDive

<!-- 
author: massimiliano.canosi
place: Milan
date: 2023-09-02 
-->

In [24]:
import requests 
import json 
import requests_with_caching

In [25]:
movie_title = 'your_title_here'

In [34]:
def get_movies_from_tastedive(movie_title):
    base_url = 'https://tastedive.com/api/similar'
    request = {'q': movie_title, 'type': 'movies', 'limit': 5}
    response = requests_with_caching.get(base_url, params = request)
    return (response.json())


get_movies_from_tastedive - Example

In [ ]:
# {
#      "Similar": {
#          "Info": [
#              {
#                  "Name": "Black Panther",
#                  "Type": "movie"
#              }
#          ],
#          "Results": [
#              {
#                  "Name": "Captain Marvel",
#                  "Type": "movie"
#              },
#              {
#                  "Name": "Avengers: Infinity War",
#                  "Type": "movie"
#              },
#              {
#                  "Name": "Ant-Man And The Wasp",
#                  "Type": "movie"
#              },
#              {
#                  "Name": "The Fate Of The Furious",
#                  "Type": "movie"
#              },
#              {
#                  "Name": "Deadpool 2",
#                  "Type": "movie"
#              }
#          ]
#      }
#  } 

Movie Titles

In [122]:
def extract_movie_titles(my_dict):
    lst = [element['Name'] for element in my_dict['Similar']['Results']]
    return(lst)

extract_movie_titles - Example

In [ ]:
# ['The Startup Kids', 'Charlie Chaplin', 'Venus In Fur', 'Loving', 'The African Queen']

Related Titles

In [123]:
def get_related_titles(titles_lst):
    otp = []
    for title in titles_lst:
        mv_lst = extract_movie_titles(get_movies_from_tastedive(title))
        for movie in mv_lst:
            if movie not in otp:
                otp.append(movie)
    return(otp)

get_related_titles - Example

In [ ]:
# ['Captain Marvel', 'Avengers: Infinity War', 'Ant-Man And The Wasp', 'The Fate Of The Furious', 'Deadpool 2', 'Inhumans', 'Venom', 'American Assassin', 'Black Panther']

In [124]:
def get_movie_data(OMDb_title):
    base_url = 'http://www.omdbapi.com/'
    request = {'t': OMDb_title, 'r': 'json'}
    response = requests_with_caching.get(base_url, params = request)
    return(response.json())

get_movie_data - Example

In [ ]:
# OMDb_dict = {
#     'Title': 'Baby Mama',
#     'Year': '2008',
#     'Rated': 'PG-13',
#     'Released': '25 Apr 2008',
#     'Runtime': '99 min',
#     'Genre': 'Comedy, Romance',
#     'Director': 'Michael McCullers',
#     'Writer': 'Michael McCullers',
#     'Actors': 'Amy Poehler, Tina Fey, Greg Kinnear, Dax Shepard',
#     'Plot': 'A successful, single businesswoman who dreams of having a baby discovers she is infertile and hires a working class woman to be her unlikely surrogate.',
#     'Language': 'English',
#     'Country': 'USA',
#     'Awards': '1 win & 5 nominations.',
#     'Poster': 'https://m.media-amazon.com/images/M/MV5BMTYwNTc1Nzk1N15BMl5BanBnXkFtZTcwNjE4OTI2MQ@@._V1_SX300.jpg',
#     'Ratings': [
#         {'Source': 'Internet Movie Database', 'Value': '6.0/10'},
#         {'Source': 'Rotten Tomatoes', 'Value': '64%'},
#         {'Source': 'Metacritic', 'Value': '55/100'}
#     ],
#     'Metascore': '55',
#     'imdbRating': '6.0',
#     'imdbVotes': '37,454',
#     'imdbID': 'tt0871426',
#     'Type': 'movie',
#     'DVD': '09 Sep 2008',
#     'BoxOffice': '$60,269,340',
#     'Production': 'Universal Pictures',
#     'Website': 'http://www.babymamamovie.net/',
#     'Response': 'True'  
# }

In [68]:
def get_movie_rating(OMDb_dict):
    for i in OMDb_dict['Ratings']:
        if i['Source'] == 'Rotten Tomatoes' and i['Value'] != '':
            rating = i['Value'][:-1]
            break
        else:
            rating = '0'
    return(int(rating))

In [2]:
def get_sorted_recommendations(final_lst):
    reccomendations = get_related_titles(final_lst)
    movie_rating_pair = {}
    for movie in reccomendations:
        if movie not in movie_rating_pair:
            movie_rating_pair[movie] = get_movie_rating(get_movie_data(movie))
    sorting_by_value = lambda x: x[1]
    sorting_by_alph = lambda x: x[0][0]
    otp = [i[0] for i in sorted(movie_rating_pair.items(), key = lambda x: (sorting_by_value(x), sorting_by_alph(x)), reverse=True)]
    return (otp)